In [ ]:
import numpy as np # type: ignore
import random
import copy
import matplotlib.pyplot as plt # type: ignore

In [ ]:
class Problem:
    def __init__(self):
        pass


In [ ]:
def decode(chromosome, problem: Problem):
    pass

def get_fitness(x):
    pass

In [ ]:
class Individual:
    def __init__(self):
        self.chromosome = None
        self.fitness = None

    def genIndi(self, problem : Problem):
        self.chromosome
        # pass
    def clone(self):
        return copy.deepcopy(self)
    
    def cal_fitness(self, problem):
        self.fitness = get_fitness(self.chromosome)
 
    def __repr__(self):
        return f"chromosome={self.chromosome}, fitness={self.fitness}" 


In [ ]:
def crossover(I_k, V_k, CR):
    O_k = Individual()
    O_k.chromosome = np.zeros(len(I_k.chromosome))
    j = np.random.randint(0, len(I_k.chromosome))
    for i in range(len(I_k.chromosome)):
        if np.random.rand() <= CR or i == j:
            O_k.chromosome[i] = V_k.chromosome[i]
        else:
            O_k.chromosome[i] = I_k.chromosome[i]
    O_k.chromosome = np.clip(O_k.chromosome, 0, 1)
    return O_k.clone()

def mutation(parent: list, F):
    V_k = Individual()
    a = np.zeros(len(parent[0].chromosome))
    b = np.zeros(len(parent[0].chromosome))

    for i in range(len(parent)):
        if i % 2:
            a += parent[i].chromosome
        else:
            b += parent[i].chromosome
            
    V_k.chromosome = parent[-1].chromosome + F * (a - b)
    V_k.chromosome = np.clip(V_k.chromosome, 0, 1)
    return V_k.clone()

In [ ]:
class Population:
    def __init__(self, pop_size, problem : Problem):
        self.pop_size = pop_size
        self.list_indi = []
        self.problem = problem
    
    def genPop(self):
        for i in range(self.pop_size):
            indi = Individual()
            indi.genIndi(self.problem)
            indi.cal_fitness(self.problem)
            self.list_indi.append(indi)

    def __str__(self):
        result = f"pop_size = {self.pop_size}\n"
        result += "\n".join(str(indi) for indi in self.list_indi)
        return result

In [ ]:
class Selection:
    @staticmethod
    def rand(pop: list, curr_index, y):
        indices = list(range(len(pop)))
        indices.remove(curr_index)
        res_index = random.sample(indices, 2 * y + 1)
        return [pop[i] for i in res_index]
    
    @staticmethod
    def best(pop: list, curr_index, y):
        indices = list(range(len(pop)))
        best_index = max(range(len(pop)), key=lambda i: pop[i].fitness)
        indices.remove(curr_index)
        if best_index != curr_index:
            indices.remove(best_index)
        res_index = random.sample(indices, 2 * y)
        res_index.append(best_index)
        return [pop[i] for i in res_index]
    
    @staticmethod
    def current_to_best(pop: list, curr_index, y):
        indices = list(range(len(pop)))
        best_index = max(range(len(pop)), key=lambda i: pop[i].fitness)
        indices.remove(curr_index)
        if best_index != curr_index:
            indices.remove(best_index)
        res_index = random.sample(indices, 2 * y)
        res_index.insert(0, curr_index)
        res_index.insert(0, best_index)
        res_index.append(curr_index)
        return [pop[i] for i in res_index]

In [ ]:
def survival_selection(xj, uj):
    if xj.fitness > uj.fitness:
        return xj.clone()
    else:
        return uj.clone()

In [ ]:
def DE(problem, pop_size, max_gen, F, CR):
    pop = Population(pop_size, problem)
    pop.genPop()
    history = []
    for i in range(max_gen):
        best = max(pop.list_indi, key=lambda indi: indi.fitness)
        for k, I_k in enumerate(pop.list_indi):
            parent = Selection.best(pop.list_indi, k, 5)
            V_k = mutation(parent, F)
            O_k = crossover(V_k, I_k, CR)
            O_k.cal_fitness(problem)
            pop.list_indi[k] = survival_selection(I_k, O_k)
        history.append(best.fitness)
    solution = max(pop.list_indi, key=lambda indi: indi.fitness)
    return history, solution


In [ ]:
# setup
problem = Problem()
# pass
pop_size = 200
max_gen = 2000
Pc = 0.9
Pm = 0.2

# start
fitness_history, solution = DE(problem, pop_size, max_gen, Pc, Pm)

#show
for i in range(len(fitness_history)):
    print(f"Generation {i}, bestfitness = {fitness_history[i]:.2f}")


In [ ]:
#show
np.set_printoptions(precision=2, suppress=True)
print("solution:")
print(decode(solution.chromosome, problem))
print(f"{solution.fitness:.2f}")

In [ ]:
generations = list(range(len(fitness_history)))
plt.figure(figsize=(10, 5))
plt.plot(generations, fitness_history, marker='o', linestyle='-', color='b', label='Best Fitness')
plt.xlabel("Generation")
plt.ylabel("Best Fitness")
plt.title("Fitness Progress Over Generations")
plt.legend()
plt.grid(True)
plt.show()